**Importing Function Packages**



In [1]:
#@title
import numpy as np
import math
import cmath
from functools import reduce
from copy import deepcopy
import matplotlib.pyplot as plt
import numba as nb

**Functions of quantum gates**

In [2]:
#@title
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]], dtype = 'complex_')
X = np.array([[0,1],[1,0]], dtype = 'complex_')
I = np.array([[1,0],[0,1]], dtype = 'complex_')
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]], dtype = 'complex_')

def UnitGate(q, matrix, q0):
  qubit = deepcopy(q)
  for i in range(len(qubit)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r = 0
    else:
      r = 1
    qubit[i] = q[i|(1<<q0)] * matrix[r][1] + q[(i|(1<<q0))^(1<<q0)] * matrix[r][0]
  return qubit

def UnitGatetwo(q,matrix,q1,q0):
  qubit = deepcopy(q)
  for i in range(len(q)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r1 = 0
    else:
      r1 = 1
    if (i %(1<<(q1+1))) < (1<<(q1)):
      r2 = 0
    else:
      r2 = 1
    qubit[i] = q[(i|(1<<q0))|(1<<q1)] * matrix[2*r2 +r1][3]  \
    + q[((i|(1<<q0))|(1<<q1))^(1<<q0)] * matrix[2*r2 +r1][2] \
    + q[((i|(1<<q0))|(1<<q1))^(1<<q1)] * matrix[2*r2 +r1][1] \
    + q[(((i|(1<<q0)|(1<<q1))^(1<<q1))^(1<<q0))] * matrix[2*r2 +r1][0]
  return qubit

def inttobit(n, i):
    return list(map(int, list(format(i, "0{0}b".format(n)))))

**QFT** 



In [95]:
def CROT(n):
  return np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,cmath.exp((1j*cmath.pi)/2**(n-1))]])

def CROTminus(n):
  return np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,cmath.exp((-1j*cmath.pi)/2**(n-1))]])  

def powcomplex(complexmatrix):
  return (complexmatrix * (complexmatrix.real - 1j*complexmatrix.imag)).real

SWAP = CNOT = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]], dtype = 'complex_')
def SWAPQFT(qubit, n):
  for i in range(n//2 ):
    qubit = UnitGatetwo(qubit, SWAP, i, n-i-1)
  return qubit

def QFT1(qubit,n):
  if n == 0:
    return qubit
  else:
    qubit = UnitGate(qubit,H,n-1)
    if n != 1:
      for i in range(n-1):
        qubit = UnitGatetwo(qubit, CROT(n-i),i ,n-1)
    return QFT1(qubit,n-1)

def QFT(qubit,n):
  return(SWAPQFT(QFT1(qubit,n), n))


def ff(qubit, n, k):
  if k == n+1:
    return qubit
  else:
    if k != 1:
      for i in range(k-1):
        qubit = UnitGatetwo(qubit, CROTminus(2+i),k-2-i ,k-1)
    qubit = UnitGate(qubit,H,k-1)
    return ff(qubit,n,k+1)


def inversQFT(qubit,n):
  return ff((SWAPQFT(qubit,n)),n,1)

In [94]:
n = int(input())
q = np.zeros((2**(n)),dtype = 'complex_')
condition = int(input())
q[condition] = 1
print(q)
qubitQFT = QFT(q,n)
print(qubitQFT)
print(inversQFT(qubitQFT,n))

2
3
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
[ 5.000000e-01+0.j  -3.061617e-17-0.5j -5.000000e-01+0.j
  3.061617e-17+0.5j]
1
2
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
